# Assignment 2 - Convolutional Neural Network

In [8]:
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import numpy as np
import torch

def load_dataset(train_path=r"WF-data/train", test_path=r"WF-data/test"):

    transform = transforms.Compose([transforms.Resize([105, 78]),
                                    transforms.CenterCrop(size=[60, 30]),
                                    transforms.ToTensor()])
    # target_transform = {"ng": 0, "ok": 1}

    train_dataset = datasets.ImageFolder(train_path, transform=transform, target_transform=None)
    test_dataset = datasets.ImageFolder(test_path, transform=transform, target_transform=None)

    return train_dataset, test_dataset


train_dataset, test_dataset = load_dataset()
print(f"Datapoints for training is {len(train_dataset)} and for test is {len(test_dataset)}")

Datapoints for training is 136 and for test is 34


In [ ]:
# Displaying one sample image from the ImageFolder dataset

img, labels = train_dataset[100][0], train_dataset[100][1]
plt.imshow(img.permute(1,2,0))
plt.title(labels)
plt.axis(False)

The Convolutional Neural Network is based on the VGG architecture, which is a good model

In [ ]:
# Let's build a CNN!
import torch.nn as nn
    
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes=2):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
                                    nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
                                    nn.BatchNorm2d(32),
                                    nn.ReLU())
        self.layer2 = nn.Sequential(
                                    nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
                                    nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU())
        self.layer4 = nn.Sequential(
                                    nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
                                    nn.BatchNorm2d(128),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.fc1 = nn.Sequential(nn.Dropout(0.5), nn.Linear(7*15*128, 4096), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Dropout(0.5), nn.Linear(4096, 128), nn.ReLU())
        self.dense = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return self.dense(out)

In [ ]:
from torch.utils.data import DataLoader

dev = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# Create data loaders
batch_size = 5
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Creating the Early Stopping class
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True


# Define model, loss function, optimizer and other hyper-parameters
import torch.optim as optim

model = ConvNeuralNet().to(dev) # Based on VGG architecture
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
early_stopping = EarlyStopping(tolerance=2, min_delta=2)

In [ ]:
# Train the model
from tqdm.auto import tqdm as tqdm

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    test_correct = 0

    for _, (images, labels) in enumerate(tqdm(train_loader, position=0, leave=True, ascii=False)):
        
        images = images.to(dev)
        labels = labels.to(dev)

        optimizer.zero_grad() # Zero out the gradients for initialisation
        outputs = model(images) # Compute output for Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward() # Perform backpropagation step
        optimizer.step() # Update the weights

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        del images, labels, outputs 

    train_loss /= len(train_loader)
    train_accuracy = train_correct / len(train_dataset)
    

   # Validation Process
    with torch.no_grad(): 
        valid_correct = 0
        total = 0
        valid_loss = 0
        model.eval()
        for _, (images, labels) in enumerate(tqdm(test_loader, position=0, leave=False, ascii=False)):
            images = images.to(dev)
            labels = labels.to(dev)
            
            outputs = model(images)

            # _, predicted = torch.max(outputs.data, 1)
            # valid_correct += (predicted == labels).sum().item()
            valid_loss += criterion(outputs, labels).item()
            

    # Early Stopping for avoiding overfit
    print(f'Epoch {epoch+1}: Train Loss= {train_loss:.3f}, Train Accuracy: {train_accuracy:.3f}, Validation Loss = {valid_loss / len(test_loader):.2f}') 

    if early_stopping(valid_loss, train_loss):
        print("We are at epoch:", epoch)
        break
print('Finished training')


# Add your own test here